In [1]:
import numpy as np
import pandas as pd

import torch
from torch import nn,optim

import os

d:\Anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 数据

In [5]:
data_dir = "E:/DATA/feedback-prize-effectiveness/"

In [10]:
df1 = pd.read_csv(data_dir+"train.csv")
print(df1.head(10))
print(df1.iloc[0]['discourse_text'])
#essay_id   txt文件名
#discourse_id   段落的id，应该没用

   discourse_id      essay_id  \
0  0013cc385424  007ACE74B050   
1  9704a709b505  007ACE74B050   
2  c22adee811b6  007ACE74B050   
3  a10d361e54e4  007ACE74B050   
4  db3e453ec4e2  007ACE74B050   
5  36a565e45db7  007ACE74B050   
6  fb65fe816ba3  007ACE74B050   
7  4e472e2584fa  007ACE74B050   
8  28a94d3ee425  007ACE74B050   
9  d226f06362f5  00944C693682   

                                      discourse_text        discourse_type  \
0  Hi, i'm Isaac, i'm going to be writing about h...                  Lead   
1  On my perspective, I think that the face is a ...              Position   
2  I think that the face is a natural landform be...                 Claim   
3  If life was on Mars, we would know by now. The...              Evidence   
4  People thought that the face was formed by ali...          Counterclaim   
5  though some say that life on Mars does exist, ...              Rebuttal   
6  It says in paragraph 7, on April 5, 1998, Mars...              Evidence   
7  Everyone 

In [9]:
df2 = pd.read_csv(data_dir+"test.csv")
print(df2.head())
print(df2.iloc[0]['discourse_text'])

   discourse_id      essay_id  \
0  a261b6e14276  D72CB1C11673   
1  5a88900e7dc1  D72CB1C11673   
2  9790d835736b  D72CB1C11673   
3  75ce6d68b67b  D72CB1C11673   
4  93578d946723  D72CB1C11673   

                                      discourse_text discourse_type  
0  Making choices in life can be very difficult. ...           Lead  
1  Seeking multiple opinions can help a person ma...       Position  
2                     it can decrease stress levels           Claim  
3             a great chance to learn something new           Claim  
4               can be very helpful and beneficial.           Claim  
Making choices in life can be very difficult. People often ask for advice when they can not decide on one thing. It's always good to ask others for their advice when making a choice. When you have multiple opinions you have the ability to make the best choice for yourself. 


In [12]:
with open(data_dir+"/train/000E6DE9E817.txt") as f:
    print(f.read())
print("-------")
with open(data_dir+"/train/00B144412785.txt") as f:
    print(f.read())

Dear: Principal

I am arguing against the policy change because even though there are some children out there that really needs help with their academic work, that does not mean that only because they have a c average that would not let them enjoy their sports or other activities unless they've a B average.

Sometimes teachers or even principal needs to consider that we should give the help that any student should have. Also this may consider student self as steam. Meaning student would start to feel sad nervous, and not wanting to go to school because of the reason they have a low averages and they can not participate in other activities or sports. The fact that there are children that would want to enjoy many good things the school is actually giving it to them.

We would want to make changes as, "like to be a better person for a better tomorrow" This supports the idea of having have many good thoughts and incasing your work as much as possible. In some situation like arguing we shou

# rnn

## 数据基本处理
1. 一般语言处理中对全部数据只会取常用的n个词，在此之外的词是不认识的，即先有一本字典
2. 用one-hot编码时，每个词都表示为长n的向量，其中只有一个值是1，其余全是0。比如字典的第i（0开始）个单词其编码中1值的下标是i

## embedding
1. one_hot编码稀疏，所以考虑用稠密向量表示词，比如只用长为m<n的向量表示n个单词，其中单词间关系可以体现在向量间关系中，比如$\vec{男}+\vec{国王}=\vec{皇帝}$
2. 可由embedding层实现这个，embedding记录了一个(n,m)的矩阵，每行都是一个单词的稠密向量，作用是one-hot编码的向量按照其1值的下标i访问这个矩阵第i行，取出这行向量作为新输入
3. 具体使用看下面代码说明
4. embedding可由自行训练出，也可预加载预训练参数。使用预训练参数时，冻结此层

## 初步结果
将batch_size\*len_sentences\*n变为batch_size\*len_sentences\*m

## rnn具体流程
1. 首先初始化hadden_input为全0
2. 对每个词，其都会和当前的hadden_input一起进入网络(cat或add)进行一步（liner、tanh激活）运算，所得的输出作为新的hadden_input与下一个词的向量一起进入网络（cat或add）
3. cat（最后一维）的话，需要截断，或者另外卷积一次获得新hadden_input
4. pytorch的rnn为x[i]通过一liner，hidden通过一liner，两个结果相加经Tanh激活，结果作为x[i]和新hidden，具体见下面代码

In [3]:
# rnn内部细节
batch_size = 4
sentence_len = 20 #一句话20个词
words_num = 100 #字典记录了100个词
words_len = 8 #字典的每个词向量长8
X = torch.randint(0,words_num,[batch_size,sentence_len]) #注意输入不为one-hot，只是每个词的字典序号，比如[3,2,10]表示一句话。int
y = torch.as_tensor([[0.,1],[0,1],[1,0],[1,0]])
#print(X)

hidden_layer_num = 1 #多少个hidden用于循环，即多少个循环部分
class MyRNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.hidden_size = 128
        self.embedding_layer = nn.Embedding(words_num,words_len,padding_idx=0) #100*8,padding_idx为输入长度不够时填充的字典词序号

        #一个循环节
        self.hidden_layer_x = nn.Linear(words_len,self.hidden_size) #rnn关键部分
        self.hidden_layer_h = nn.Linear(self.hidden_size,self.hidden_size) #给h用

        #分类器
        self.out_layer = nn.Linear(self.hidden_size,2)
        self.activation_layer = nn.Softmax(dim=-1) #dim=0表示a[i][j][k]按i方向的几个数一起算

    def __init_hidden__(self):
        return torch.zeros([hidden_layer_num,batch_size,self.hidden_size])

    def forward(self,x):
        self.hidden = self.__init_hidden__()

        print(f"before embedding:{x.shape}") #[4, 20]
        x = self.embedding_layer(x)
        print(f"after embedding:{x.shape}") #[4, 20, 8]
        print("-------------")

        out = torch.zeros([sentence_len, batch_size, self.hidden_size])

        #rnn部分
        # 为了更好计算，将数据x变形为为len_sencentces*batch_size*words_num
        # 即x[0]为各句子首单词
        x = x.transpose(0,1)
        print(f"rnn input:{x.shape}") #[20, 4, 128]
        for i in range(x.shape[0]):
            a1 = self.hidden_layer_x(x[i])
            a2 = self.hidden_layer_h(self.hidden[0])

            out[i] = self.hidden[0] = nn.Tanh()(a1+a2)
        print(f"rnn out:{out.shape}")
        print(f"rnn outh:{self.hidden[0].shape}")
        print("-----------")
        
        #分类器
        o = self.hidden[0]
        o = self.out_layer(o)
        o = self.activation_layer(o)
        print(f"finally shape:{o.shape}") #[4, 2]
        print(o)

        return o

rnn = MyRNN()
out = rnn(X)

before embedding:torch.Size([4, 20])
after embedding:torch.Size([4, 20, 8])
-------------
rnn input:torch.Size([20, 4, 8])
rnn out:torch.Size([20, 4, 128])
rnn outh:torch.Size([4, 128])
-----------
finally shape:torch.Size([4, 2])
tensor([[0.5058, 0.4942],
        [0.5616, 0.4384],
        [0.4322, 0.5678],
        [0.4265, 0.5735]], grad_fn=<SoftmaxBackward0>)


## pytorch的rnn
- 首先就是输入是batch_size\*len_sencentces\*words_num  
所以Embedding算是预处理部分，如果需要训练则？？？？？？？？
- 主要公式$$h_t = \tanh(x_t W_{ih}^T + b_{ih} + h_{t-1}W_{hh}^T + b_{hh})$$

In [9]:
batch_size = 2
sentence_len = 5
words_num = 10
words_len = 8

X = torch.randint(0,words_num,[batch_size,sentence_len])

X = nn.Embedding(words_num,words_len)(X)
#X = X.transpose(0,1) #batch_size放在第二维，则batch_first设置为False
print(X.shape)

hidden_size=32
num_layers=1
H = torch.zeros([num_layers,batch_size,hidden_size]) #可以不设置，则默认为0，这是单向rnn
#HH = torch.zeros([num_layers*2,batch_size,hidden_size]) #双向rnn，需要RNN中设置bidirectional=True

rnn = nn.RNN(
            input_size=words_len,

            #hidden_size，num_layers 都是对网络的设置，与输入数据无关，设置相对自由
            hidden_size=hidden_size,     # hidden层大小
            num_layers=num_layers,       # n个rnn层

            batch_first=True, #True则输入输出的batch在第一维，否则在第二维（参照上面MyRNN在hidden前的变形）

            bidirectional=False, #是否双向rnn
        )

out,outh = rnn(X,H)
print(out.shape,outh.shape)

torch.Size([2, 5, 8])
torch.Size([2, 5, 32]) torch.Size([1, 2, 32])


In [10]:
for i in rnn.named_parameters():
    print(i[0],i[1].shape,type(i[1]))

weight_ih_l0 torch.Size([32, 8]) <class 'torch.nn.parameter.Parameter'>
weight_hh_l0 torch.Size([32, 32]) <class 'torch.nn.parameter.Parameter'>
bias_ih_l0 torch.Size([32]) <class 'torch.nn.parameter.Parameter'>
bias_hh_l0 torch.Size([32]) <class 'torch.nn.parameter.Parameter'>


## 验证

In [3]:
torch.manual_seed(43)

batch_size = 2
sentence_len = 5
words_num = 10
words_len = 4

hidden_size=8
num_layers=1

X = torch.randint(0,words_num,(batch_size,sentence_len))
em = nn.Embedding(words_num,words_len)
X = em(X)
print(X.shape)

print('pytorch部分')
rnn = nn.RNN(
            input_size=words_len,

            hidden_size=hidden_size,
            num_layers=num_layers,

            batch_first=True,
        )
out,outh = rnn(X)
print(out.shape,out[0,0,:],outh.shape,outh[0,0,:])
print('-------------------')


print('MyCNN部分')
class MyRNN(nn.Module):
    def __init__(self):
        super().__init__()


        self.hidden_layer_x = nn.Linear(words_len,hidden_size) #rnn关键部分
        self.hidden_layer_h = nn.Linear(hidden_size,hidden_size) #h用

        params = list(rnn.parameters())

        self.hidden_layer_x.weight = params[0]
        self.hidden_layer_h.weight = params[1]
        self.hidden_layer_x.bias = params[2]
        self.hidden_layer_h.bias = params[3]

    def __init_hidden__(self):
        return torch.zeros([num_layers,batch_size,hidden_size])

    def forward(self,x):
        self.hidden = self.__init_hidden__()

        out = torch.zeros([sentence_len,batch_size,hidden_size])

        x = x.transpose(0,1)
        for i in range(x.shape[0]):
            a1 = self.hidden_layer_x(x[i])
            a2 = self.hidden_layer_h(self.hidden[0])
            self.hidden[0] = nn.Tanh()(a1+a2)


            out[i] = self.hidden[0]

        return out.transpose(0,1)

myrnn = MyRNN()
out = myrnn(X)
print(out.shape,out[0,0,:],myrnn.hidden.shape,myrnn.hidden[0,0,:])

torch.Size([2, 5, 4])
pytorch部分
torch.Size([2, 5, 8]) tensor([ 0.0445,  0.4097,  0.3055, -0.1414, -0.7245,  0.5351, -0.2224, -0.1610],
       grad_fn=<SliceBackward0>) torch.Size([1, 2, 8]) tensor([-0.0913,  0.1614, -0.0179, -0.8154,  0.3454, -0.7803, -0.3056,  0.3397],
       grad_fn=<SliceBackward0>)
-------------------
MyCNN部分
torch.Size([2, 5, 8]) tensor([ 0.0445,  0.4097,  0.3055, -0.1414, -0.7245,  0.5351, -0.2224, -0.1610],
       grad_fn=<SliceBackward0>) torch.Size([1, 2, 8]) tensor([-0.0913,  0.1614, -0.0179, -0.8154,  0.3454, -0.7803, -0.3056,  0.3397],
       grad_fn=<SliceBackward0>)


# LSTM
- rnn因为激活层是Tanh，显然，当前信息几乎不会对很远的计算产生影响，即只能短期记忆
- lstm中引入可以选择“记忆”和“当前”的信息对当前输出的占比  
  
nn.LSTM为什么有两个参数矩阵？有一个是4个矩阵组合，这四个矩阵是给四个部分用的

In [2]:
torch.manual_seed(43)

batch_size = 2
sentence_len = 5
words_num = 10
words_len = 4

hidden_size=4
num_layers=1

X = torch.randint(0,words_num,(batch_size,sentence_len))
em = nn.Embedding(words_num,words_len)
X = em(X)
print(X.shape)

print('pytorch部分')
lstm = nn.LSTM(
            input_size=words_len,

            hidden_size=hidden_size,
            num_layers=num_layers,

            batch_first=True,
        )
H = torch.zeros([num_layers,batch_size,hidden_size])
C = torch.zeros([num_layers,batch_size,hidden_size])
out,(outh,outc) = lstm(X,(H,C)) #H、C默认为0
print(out.shape,out[0,0,:],outh.shape,outc.shape)
print('-------------------')


print('MyLSTM部分')
class MyLSTM(nn.Module):
    def __init__(self):
        super().__init__()

        self.

        self.hidden_layer_xh = nn.Linear(hidden_size*4,hidden_size)

        params = list(lstm.parameters())
        self.hidden_layer_xh1.weight = params[0]
        self.hidden_layer_xh2.weight = params[1]
        self.hidden_layer_xh2.bias = params[2]
        self.hidden_layer_xh2.bias = params[3]

    def __init_H_C__(self):
        return torch.zeros([1,batch_size,hidden_size]),torch.zeros([1,batch_size,hidden_size])


    def forward(self,x):
        self.H,self.C = self.__init_H_C__()
        #H是输出
        #C是记忆

        x = x.transpose(0,1)
        for i in torch.arange(x.shape[0]):
            hidden1 = self.hidden_layer_xh1(x[i]+self.H)
            hidden2 = self.hidden_layer_xh2(x[i]+self.H)
			
            cp = nn.Sigmoid()(hidden1) #C即记忆的保留
            self.C = self.C*cp+nn.Tanh()(hidden2)*(1-cp) #更新C
            self.H = nn.Tanh()(self.C)*nn.Sigmoid()(hidden1) #生成新的输出

        x = x.transpose(0,1)
        return x

mylstm = MyLSTM()
out = mylstm(X)
print(out.shape,out[0,0,:],mylstm.C.shape,mylstm.H.shape)

torch.Size([2, 5, 4])
pytorch部分
torch.Size([2, 5, 4]) tensor([-0.2213,  0.1023, -0.2557, -0.2017], grad_fn=<SliceBackward0>) torch.Size([1, 2, 4]) torch.Size([1, 2, 4])
-------------------
MyLSTM部分


AttributeError: 'MyLSTM' object has no attribute 'hidden_layer_xh1'

In [5]:
for i in lstm.parameters():
    print(i.shape)

torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16])
torch.Size([16])
